In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import re

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

df = pd.DataFrame(None, columns=['name','time','portions','difficulty','description', 'ingredients','steps', 'website', 'url', 'image_url'])

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Martin\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [5]:
drink_links = []
for page in range(1,5):
    driver.get(f"https://www.bbcgoodfood.com/recipes/collection/cocktail-recipes?page={page}")
    for item in driver.find_elements(by=By.CSS_SELECTOR, value='.card--inline .link')[::2]:
        drink_links.append(item.get_attribute('href'))
assert len(drink_links) == 90

In [ ]:
website = 'bbc'
from time import sleep
def scrape(url,df):
    driver.get(url)
    sleep(5)
    try:
        driver.find_element(by=By.XPATH, value='//*[@id="qc-cmp2-ui"]/div[2]/div/button[1]').click()
    except NoSuchElementException:
        pass
    try:
        driver.find_element(by=By.CLASS_NAME, value='button reject').click()
    except NoSuchElementException:
        pass
    drink_name = driver.find_element(by = By.CSS_SELECTOR, value=".heading-1").text.lower()
    time = driver.find_element(by = By.CSS_SELECTOR, value="time").text
    portions = driver.find_element(by = By.CSS_SELECTOR, value=".post-header__servings .icon-with-text__children").text.split()[1]
    difficulty = driver.find_element(by = By.CSS_SELECTOR, value=".post-header__skill-level .icon-with-text__children").text
    description = driver.find_element(by = By.CSS_SELECTOR, value=".pr-xxs p").text
    ingredients = driver.find_element(by = By.XPATH, value='//*[@id="__next"]/div[3]/main/div/div/div[1]/div[1]/div[2]/div[2]/div/section[1]/section').text.split('\n')
    steps = driver.find_element(by = By.CSS_SELECTOR, value=".mb-lg.col-lg-6").text.split('\n')[::2][1:]
    
    image = driver.find_element(by = By.CSS_SELECTOR, value=".post-header-image .image__img")
    image_url = image.get_attribute('src')

    image_url_clean = re.sub('\?.*','',image_url)
    
    result = {
        'name': drink_name,
        'time': time,
        'portions': portions,
        'difficulty': difficulty,
        'description': description,
        'ingredients': ingredients,
        'steps': steps,
        'url': url,
        'website': website,
        'image_url': image_url_clean
    }
    df = df.append(result, ignore_index=True)

    with open(f'images/{website}/{drink_name}.png', 'wb') as file:
        file.write(image.screenshot_as_png)
    return df

for url in drink_links:
    try:
        df= scrape(url, df)
    except:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        df = scrape(url,df)

df.to_csv('./bbc.csv', index=False)